In [1]:
import tensorflow as tf

def corr2d(x, k):
    h, w = k.shape
    y = tf.Variable(tf.zeros((x.shape[0] - h + 1, x.shape[1] - w + 1)))
    for i in range(y.shape[0]):
        for j in range(y.shape[1]):
            y[i, j].assign(tf.reduce_sum(x[i: i + h, j: j + w] * k))
    return y

In [2]:
x = tf.constant([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
k = tf.constant([[0.0, 1.0], [2.0, 3.0]])
corr2d(x, k)

<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[19., 25.],
       [37., 43.]], dtype=float32)>

In [3]:
class Conv2d(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()
        
    def build(self, kernel_size):
        initilizer = tf.random_normal_initializer()
        self.weight = self.add_weight(name='w', shape=kernel_size, 
                                     initialzer=initializer)
        self.bias = self.add_weight(name='b', shape=(1,), 
                                   initializer=initializer)
        
    def call(self, inputs):
        return corr2d(inputs, self.weight) + self.bias

In [4]:
x = tf.Variable(tf.ones((6, 8)))
x[:, 2:6].assign(tf.zeros(x[:, 2:6].shape))
x

<tf.Variable 'Variable:0' shape=(6, 8) dtype=float32, numpy=
array([[1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.]], dtype=float32)>

In [5]:
k = tf.constant([[1.0, -1.0]])

In [6]:
y = corr2d(x, k)
y

<tf.Variable 'Variable:0' shape=(6, 7) dtype=float32, numpy=
array([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.]], dtype=float32)>

In [7]:
corr2d(tf.transpose(x), k)

<tf.Variable 'Variable:0' shape=(8, 5) dtype=float32, numpy=
array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]], dtype=float32)>

In [8]:
conv2d = tf.keras.layers.Conv2D(1, (1, 2), use_bias=False)

x = tf.reshape(x, (1, 6, 8, 1))
y = tf.reshape(y, (1, 6, 7, 1))

y_hat = conv2d(x)
for i in range(10):
    with tf.GradientTape(watch_accessed_variables=False) as g:
        g.watch(conv2d.weights[0])
        y_hat = conv2d(x)
        l = (abs(y_hat - y)) ** 2
        update = tf.multiply(3e-2, g.gradient(l, conv2d.weights[0]))
        weights = conv2d.get_weights()
        weights[0] = conv2d.weights[0] - update
        conv2d.set_weights(weights)
        if (i + 1) % 2 == 0:
            print(f'batch {i+1}, loss {tf.reduce_sum(l):.3f}')

batch 2, loss 13.524
batch 4, loss 4.616
batch 6, loss 1.736
batch 8, loss 0.685
batch 10, loss 0.276


In [9]:
tf.reshape(conv2d.get_weights()[0], (1, 2))

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[ 0.9358542, -1.0438281]], dtype=float32)>